In [2]:
import os
import pandas as pd
import seaborn as sns
import datetime

Считывание данных

In [ ]:
#А теперь считаем и объединим информацию по пользователям и feed_data с условием
#чтобы для каждого пользователя количество действий было равно определенному числу

In [4]:
user_behavior = pd.read_sql(
    """SELECT u.*, x.*
    FROM user_data u
    LEFT JOIN LATERAL (SELECT *,
    ROW_NUMBER() OVER(PARTITION BY user_id) AS rownum
    FROM feed_data) x on u.user_id = x.user_id
    WHERE x.rownum <= 5""",
       con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
         "postgres.lab.karpov.courses:6432/startml")

In [3]:
user_data = pd.read_sql(
    """SELECT *
    FROM user_data
    """,
       con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
         "postgres.lab.karpov.courses:6432/startml")

In [5]:
user_behavior.head()

,user_id,gender,age,country,city,exp_group,os,source,timestamp,user_id,post_id,action,target,rownum
0,200,1,34,Russia,Degtyarsk,3,Android,ads,2021-10-29 19:05:49,200,5593,view,0,1
1,200,1,34,Russia,Degtyarsk,3,Android,ads,2021-10-29 19:08:06,200,260,view,0,2
2,200,1,34,Russia,Degtyarsk,3,Android,ads,2021-10-29 19:08:58,200,1167,view,0,3
3,200,1,34,Russia,Degtyarsk,3,Android,ads,2021-10-29 19:11:40,200,295,view,0,4
4,200,1,34,Russia,Degtyarsk,3,Android,ads,2021-10-29 19:12:00,200,6738,view,0,5


In [7]:

user_data.head()

,user_id,gender,age,country,exp_group,os,source
0,200,1,34,Russia,3,Android,ads
1,201,0,37,Russia,0,Android,ads
2,202,1,17,Russia,4,Android,ads
3,203,0,18,Russia,1,iOS,ads
4,204,0,36,Russia,3,Android,ads


In [8]:
from sqlalchemy import create_engine
engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)
user_data.to_sql('shatalov_dmitry_user', con=engine)

205

In [18]:
post_text_df = pd.read_sql(
    """SELECT *
    FROM post_text_df
    """,
       con="postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
         "postgres.lab.karpov.courses:6432/startml")

In [3]:
#Создадим на всякий случай копию датафрейма, чтобы какждый раз не грузить по новой данные
user_behavior_copy = user_behavior.copy()

In [4]:
#удалим технический столбец
user_behavior_copy = user_behavior_copy.drop("rownum", axis=1)

In [5]:
#Удалим повторяющиеся колонки post_id
user_behavior_copy = user_behavior_copy.loc[:,~user_behavior_copy.columns.duplicated()].copy()

In [ ]:
user_behavior_copy.head(10)

In [ ]:
user_behavior_copy.isna().sum()

In [ ]:
sns.catplot(x="gender", kind="count", col="target", data=user_behavior_copy, sharey=False)

In [ ]:
sns.catplot(x="age", kind="count", col="target", data=user_behavior_copy, sharey=False)

In [ ]:
sns.catplot(x="country", kind="count", col="target", data=user_behavior_copy, sharey=False)

In [ ]:
sns.catplot(x="os", kind="count", col="target", data=user_behavior_copy, sharey=False)

In [ ]:
sns.catplot(x="exp_group", kind="count", col="target", data=user_behavior_copy, sharey=False)

In [6]:
#для просмотров, где действие являлось лайком, проставим тоже 1 в таргет
user_behavior_copy.loc[(user_behavior_copy["action"] == "like"), "target"] = 1

In [7]:
#удалим пока что акшионы
user_behavior_copy = user_behavior_copy.drop("action", axis=1)

In [8]:
#удалим город, их слишком много, потом подумаем что делать с городом
user_behavior_copy = user_behavior_copy.drop("city", axis=1)

In [9]:

user_behavior_copy['hour'] = user_behavior_copy.timestamp.dt.hour

In [ ]:
user_behavior_copy.head()

In [10]:
user_behavior_copy['timestamp'] = user_behavior_copy['timestamp'].apply(lambda x: x.timestamp()).astype(int)

In [13]:
user_behavior_copy = user_behavior_copy.drop(["timestamp", "post_id", "target", "hour"], axis=1)

In [14]:
user_behavior_copy.head()

,user_id,gender,age,country,exp_group,os,source
0,200,1,34,Russia,3,Android,ads
1,200,1,34,Russia,3,Android,ads
2,200,1,34,Russia,3,Android,ads
3,200,1,34,Russia,3,Android,ads
4,200,1,34,Russia,3,Android,ads


In [15]:
from sqlalchemy import create_engine
engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

In [16]:
user_behavior_copy.to_sql('shatalov_dmitry_users', con=engine)

25

In [ ]:
user_behavior_copy

In [ ]:
post_text_df.nunique()

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
tf_idf = vectorizer.fit_transform(post_text_df["text"])

In [20]:
from sklearn.decomposition import TruncatedSVD

In [21]:
svd = TruncatedSVD(n_components=100)
tf_idf_svd = svd.fit_transform(tf_idf)

In [22]:
df_tf_idf_svd = pd.DataFrame(tf_idf_svd)

In [ ]:
type(df_tf_idf_svd)

In [23]:
post_text_df = pd.concat([post_text_df, df_tf_idf_svd], axis=1)

In [24]:
post_text_df.head()

,post_id,text,topic,0,1,2,3,4,5,6,...,90,91,92,93,94,95,96,97,98,99
0,1,UK economy facing major risks\n\nThe UK manufa...,business,0.386969,-0.178559,-0.090353,-0.092315,-0.124235,0.026445,-0.090919,...,-0.027707,0.009552,-0.008999,-0.013628,-0.006564,-0.010809,-0.033995,-0.037561,-0.001322,-0.013360
1,2,Aids and climate top Davos agenda\n\nClimate c...,business,0.422381,-0.203721,-0.085703,-0.002253,-0.065998,0.031487,0.019135,...,0.026562,0.026715,0.022422,0.011496,-0.030049,-0.013404,-0.006376,0.006521,0.013236,-0.014821
2,3,Asian quake hits European shares\n\nShares in ...,business,0.374896,-0.136755,-0.063838,-0.089354,-0.111561,0.020525,-0.058589,...,0.003895,-0.026901,-0.059054,0.028854,-0.045230,0.018362,-0.000664,0.047550,-0.038122,0.002763
3,4,India power shares jump on debut\n\nShares in ...,business,0.258776,-0.118369,-0.052491,-0.053808,-0.096098,0.022668,-0.033831,...,-0.029090,-0.008279,-0.047934,-0.010357,-0.006429,-0.010600,-0.018037,-0.042471,-0.019968,0.031709
4,5,Lacroix label bought by US firm\n\nLuxury good...,business,0.194117,-0.099520,-0.045275,-0.001915,-0.034148,-0.007880,-0.008969,...,-0.009430,0.012673,-0.017110,-0.011624,-0.009426,-0.003026,0.019623,-0.026298,0.011045,0.006989


In [26]:
post_text_df.to_sql("shatalov_dmitry_text", con=engine)

23

In [ ]:
user_behavior_copy = user_behavior_copy.merge(post_text_df, on=["post_id"])


In [ ]:
user_behavior_copy = user_behavior_copy.drop("text", axis = 1)

In [ ]:
user_behavior_copy = user_behavior_copy.sort_values(by="timestamp")


In [ ]:
user_behavior_copy.head()

In [ ]:
user_behavior_copy = user_behavior_copy.set_index("text")

In [ ]:
user_behavior_copy.head(5)

In [ ]:
user_behavior_copy = user_behavior_copy.set_index("user_id")

In [ ]:
X = user_behavior_copy.drop('target', axis=1)
y = user_behavior_copy['target']

In [ ]:
user_behavior_copy = user_behavior_copy.drop("post_id", axis = 1)

In [ ]:
user_behavior_copy.head()

In [ ]:
X_train = X.iloc[0 : int(X.shape[0]*0.8)]
y_train = y.iloc[0 : int(y.shape[0]*0.8)]

In [ ]:
X_test = X.iloc[int(X.shape[0]*0.8) : int(X.shape[0])]
y_test = y.iloc[int(X.shape[0]*0.8) : int(X.shape[0])]

In [ ]:
X_test.head()

In [ ]:
X.shape

In [ ]:
int(X.shape[0]*0.8)

In [ ]:
X_train.head()

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
X_test.columns[0]

In [ ]:
from sklearn.metrics import classification_report




cboost = CatBoostClassifier(auto_class_weights="SqrtBalanced")

cboost.fit(X_train,
           y_train,
           cat_features=['country', "os", "source", "topic"] )

y_pred = cboost.predict(X_test)

print(classification_report(y_test, y_pred, digits=4))

In [ ]:
cboost.feature_importances_

In [ ]:
cboost.save_model('catboost_model_3', format="cbm")

In [ ]:
user_behavior_copy.to_csv('catboost_f.csv')

In [ ]:
user.nunique()

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
def batch_load_sql(query: str) -> pd.DataFrame:
    CHUNKSIZE = 200000
    engine = create_engine(
        "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
        "postgres.lab.karpov.courses:6432/startml"
    )
    conn = engine.connect().execution_options(stream_results=True)
    chunks = []
    for chunk_dataframe in pd.read_sql(query, conn, chunksize=CHUNKSIZE):
        chunks.append(chunk_dataframe)
    conn.close()
    return pd.concat(chunks, ignore_index=True)

def load_features():
    return batch_load_sql("""SELECT * FROM "shatalov_dmitry_f3" """)

In [ ]:
df = load_features()
df

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

In [ ]:
user_behavior_copy.head()


In [ ]:
engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)


In [ ]:
user_behavior_copy.to_sql('shatalov_dmitry_f3', con=engine)

In [ ]:
user_behavior_copy.shape

In [3]:
df = pd.read_csv("catboost_f.csv")
df.head()

,Unnamed: 0,user_id,gender,age,country,exp_group,os,source,timestamp,post_id,...,90,91,92,93,94,95,96,97,98,99
0,120196,122594,0,33,Russia,1,Android,organic,1633068100,4554,...,0.037502,0.036908,-0.034453,-0.061260,-0.043897,0.016319,-0.011568,0.018723,-0.050328,0.094512
1,399987,43965,1,28,Russia,4,iOS,ads,1633068100,5747,...,-0.013138,-0.033236,-0.012336,0.019883,0.014769,0.015488,0.023121,-0.024535,-0.008846,-0.018057
2,271957,94081,1,20,Russia,3,Android,ads,1633068100,5671,...,-0.022173,-0.003069,-0.031064,0.020271,-0.018853,-0.004576,-0.045898,0.050105,0.039707,-0.022490
3,34605,136194,0,24,Russia,3,iOS,organic,1633068100,1205,...,-0.019943,-0.006733,-0.027831,-0.005570,0.001175,0.014934,-0.002055,-0.005040,-0.013210,0.025356
4,581384,141770,0,27,Russia,0,Android,organic,1633068112,583,...,-0.027622,0.010700,-0.013210,-0.004139,0.005926,0.001327,0.018616,-0.022207,0.000192,0.022039
